In [1]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.7 MB/s eta 0:00:00


 bitsandbytes accelerate for GPU based machine

In [3]:
!pip -q install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.3 MB/s eta 0:00:00


for cloud based vector db , we need client

mongodb-client

huggingface-client

In [4]:
!pip install pinecone-client==2.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.6 MB/s eta 0:00:00


In [5]:
from  langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Extract data from URL

In [6]:
URLs = [
    "https://ineuron.ai/"
]

In [7]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
!pip show numpy

Name: numpy
Version: 1.25.2
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: accelerate, albumentations, altair, arviz, astropy, autograd, bitsandbytes, blis, bokeh, bqplot, chex, cmdstanpy, contourpy, cufflinks, cupy-cuda12x, cvxpy, datascience, db-dtypes, dopamine-rl, ecos, flax, folium, geemap, gensim, gym, h5py, holoviews, hyperopt, ibis-framework, imageio, imbalanced-learn, imgaug, jax, jaxlib, langchain, langchain-community, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, opencv-contrib-python, opencv-python, opencv-python-headless, opt-einsum, optax, orbax-checkpoint, osqp, pandas, pandas-gbq, pandas-stubs, patsy, pinecone-client, plotnine, prophet, pyarrow, pycocotools, pyerfa, pymc, pytensor, python-louvain, PyW

In [9]:
data

[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,\n\nAn entire learning ecosystem at your fingertips to make learning fun.\n\nChoose from a range of tech programs and make your next big career switch.\n\nExplore Courses\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster.\n\nExperience a tech community like never seen before\n\nTake me there\n\nOur Courses\n\nView all\n\nView all\n\nSuccess Stories\n\nView all\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Kri

# Chunkins


In [10]:
text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size = 1000,
                                      chunk_overlap = 200)

In [11]:
text_chunks = text_splitter.split_documents(data)

In [12]:
len(text_chunks)

6

In [13]:
text_chunks[0]

Document(page_content='Learning with iNeuron made \xa0<>\nTake your career to the next level with industry ready programs,\nAn entire learning ecosystem at your fingertips to make learning fun.\nChoose from a range of tech programs and make your next big career switch.\nExplore Courses\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster.\nExperience a tech community like never seen before\nTake me there\nOur Courses\nView all\nView all\nSuccess Stories\nView all\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSo

# Embedding Model

for converting the chunks in vector embeddings

In [14]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
query_result = embeddings.embed_query("ineuron")

In [16]:
len(query_result)

384

In [17]:
query_result

[-0.09189119189977646,
 0.05263545736670494,
 -0.09840165823698044,
 -0.005310880020260811,
 0.03780684620141983,
 -0.00528704933822155,
 0.05091768503189087,
 0.1363333761692047,
 0.06077226996421814,
 -0.033349450677633286,
 0.03781449422240257,
 -0.09755448997020721,
 -0.030724111944437027,
 0.06230128929018974,
 -0.08405937999486923,
 0.03709800913929939,
 -0.009448325261473656,
 0.0011943514691665769,
 -0.05092259496450424,
 0.015395219437777996,
 -0.06587088853120804,
 0.08156675100326538,
 -0.04317218437790871,
 0.0146227041259408,
 -0.08008887618780136,
 -0.07103437185287476,
 0.022798754274845123,
 0.021937618032097816,
 -0.017158396542072296,
 -0.08246370404958725,
 -0.01789167895913124,
 -0.012339456006884575,
 -0.011260810308158398,
 0.014543679542839527,
 -0.017382634803652763,
 0.054021719843149185,
 0.02673119306564331,
 -0.00709928572177887,
 0.07976948469877243,
 0.011656652204692364,
 0.00036512623773887753,
 0.005530111026018858,
 -0.06251872330904007,
 -0.0380367264